# Maximisation d'influence

On cherche à modéliser la propagation d'influence dans un graphe de réseau social. On compare ici deux modèles différents : l'algorithme des cascades indépendantes (independant cascade model) et le modèle des seuils linéaires (linear thresholds model)

## Cascades indépendantes

**Initialisation :** 
Un graphe $G = (V,E)$ et un $I \subset V$ un sous ensemble de noeuds initialement infectés, $N = \emptyset$ l'ensemble des noeuds inactifs.

**Algorithme**

Tant que $I$ n'est pas vide : 

&nbsp; &nbsp; Soit $u \in I$ 

&nbsp; &nbsp; Pour $v \in voisins(u)$ : 

&nbsp; &nbsp; &nbsp; &nbsp; si $v \notin N \cup I $ : 

&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; ajouter $v$ à $I$ avec une probabilité $poids(u,v)$ 

&nbsp; &nbsp;  $I \gets I \setminus \{ u \}$

&nbsp; &nbsp;  $N \gets N \cup \{ u \}$

**Sortie**

$N$


## Seuils linéaires

**Initialisation :** 
Un graphe $G = (V,E)$ et un $I \subset V$ un sous ensemble de noeuds initialement infectés

**Algorithme**

Tant que $I$ n'est pas stable : 

&nbsp; &nbsp; Soit $u \in I$ non encore visité

&nbsp; &nbsp; Pour $v \in voisins(u)$ : 

&nbsp; &nbsp; &nbsp; &nbsp; Soit $W$ l'ensemble des voisins entrants de v infectés 

&nbsp; &nbsp; &nbsp; &nbsp; si $\sum_{v' \in W} poids(v',v) > seuil(v) $ : 

&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; $I \gets I \cup \{ v \} $

&nbsp; &nbsp;  $I \gets I \setminus \{ u \}$


**Sortie**

$I$

## Implémentation

In [1]:
import org.apache.spark._
import org.apache.spark.graphx._
import org.apache.spark.rdd.RDD

Intitializing Scala interpreter ...

Spark Web UI available at http://LAPTOP-0NFRQ5HD:4041
SparkContext available as 'sc' (version = 3.1.1, master = local[*], app id = local-1618524204852)
SparkSession available as 'spark'


import org.apache.spark._
import org.apache.spark.graphx._
import org.apache.spark.rdd.RDD


In [2]:
def independant_cascade(graph:Graph[String,Float]):Graph[String,Float]={
    def mergeMsg (m1:String, m2:String):String = {
        if(m1=="become_inactive" || m2 == "become_inactive"){return "become_inactive"}
        if (m1 == "become_infected" || m2 == "become_infected"){return "become_infected"}
        return "nothing"
    }

    def vprog(VertexId:VertexId, VD:String, A:String): String ={
        if (VD == "susceptible" && A == "become_infected"){ return "infected"}
        if(VD=="infected" && A == "become_inactive"){return "inactive"}
        return VD
    }

    def sendMsg(triplet:EdgeTriplet[String, Float]): Iterator[(VertexId, String)]={
        if (triplet.srcAttr == "infected"){
            if (scala.util.Random.nextFloat < triplet.attr){
                return Iterator((triplet.dstId,"become_infected"),(triplet.srcId,"become_inactive"))
            }
        return Iterator((triplet.srcId,"become_inactive"))
        }
        return Iterator()
    }
    
    return graph.pregel("nothing",Int.MaxValue,EdgeDirection.Out)(vprog,sendMsg,mergeMsg)
    .mapVertices((vid,vdata) => if(vdata == "infected") "inactive" else vdata)
}

independant_cascade: (graph: org.apache.spark.graphx.Graph[String,Float])org.apache.spark.graphx.Graph[String,Float]


In [3]:
type VD = (String,Float,Float)
type ED = Float
type Message = (String,Float)

def linear_threshhold(graph:Graph[VD,ED]):Graph[VD,ED]={
    def mergeMsg (m1:Message, m2:Message):Message = {
        if(m1._1=="become_inactive" || m2._1 == "become_inactive"){return ("become_inactive",0f)}
        if (m1._1 == "become_infected" && m2._1 == "become_infected"){return ("become_infected",m1._2+m2._2)}
        if (m1._1 == "become_infected" ){return m1}
        if (m2._1 == "become_infected" ){return m2}
        return ("nothing",0f)
    }

    def vprog(VertexId:VertexId, vdata:VD, m:Message): VD ={
        if (m._1 == "become_inactive"){
            return ("inactive", vdata._2, vdata._3)
        }
        if (m._1 == "become_infected"){
            val state = vdata._1
            val new_infection_rate = vdata._2 + m._2
            val threshold = vdata._3
            if (state == "susceptible" && new_infection_rate > threshold){
                return ("infected",new_infection_rate,threshold)
            }
            return (state,new_infection_rate,threshold)
        }
        return vdata
    }

    def sendMsg(triplet:EdgeTriplet[VD, ED]): Iterator[(VertexId, Message)]={
        if (triplet.srcAttr._1 == "infected"){
            return Iterator((triplet.dstId,("become_infected",triplet.attr)),(triplet.srcId,("become_inactive",0f)))

        return Iterator((triplet.srcId,("become_inactive",0f)))
        }
        return Iterator()
    }
    
    return graph.pregel(("nothing",0f),Int.MaxValue,EdgeDirection.Out)(vprog,sendMsg,mergeMsg)
}

defined type alias VD
defined type alias ED
defined type alias Message
linear_threshhold: (graph: org.apache.spark.graphx.Graph[VD,ED])org.apache.spark.graphx.Graph[VD,ED]


## Fonctions utiles

In [4]:
def normalize_graph_weights[VD](graph:Graph[VD,Float])(implicit m: ClassManifest[VD]):Graph[VD,Float] = {
    val weight_sums_per_dst = graph.triplets.map(t => (t.dstId,t.attr))
                            .reduceByKey( _ + _)
    val newEdges = graph.edges.map(e => (e.dstId,e))
         .join(weight_sums_per_dst).map{ case ((dstId,(edge,weightSum)))=>Edge(edge.srcId,edge.dstId,edge.attr/weightSum)}
    return Graph(graph.vertices,newEdges)
}

normalize_graph_weights: [VD](graph: org.apache.spark.graphx.Graph[VD,Float])(implicit m: ClassManifest[VD])org.apache.spark.graphx.Graph[VD,Float]


In [5]:
def selectVerticeWithMaxDegree(graph:Graph[Boolean,Float]):Graph[Boolean,Float] = {
    val maxDegreeVertice = graph.triplets.filter(triplet => !triplet.srcAttr && !triplet.dstAttr)
                             .map(triplet => (triplet.srcId,1))
                             .reduceByKey(_+_)
                             .reduce((v1,v2) => if(v1._2 >= v2._2 ) v1 else v2)._1
    return graph.mapVertices((vid, vdata) => vid == maxDegreeVertice || vdata)
}

val selectKVerticesWithMaxDegrees = (k:Int) => Function.chain(List.fill(k)(selectVerticeWithMaxDegree _ ))

selectVerticeWithMaxDegree: (graph: org.apache.spark.graphx.Graph[Boolean,Float])org.apache.spark.graphx.Graph[Boolean,Float]
selectKVerticesWithMaxDegrees: Int => (org.apache.spark.graphx.Graph[Boolean,Float] => org.apache.spark.graphx.Graph[Boolean,Float]) = $Lambda$2169/0x0000000100d94840@67dde44f


In [6]:
import scala.math.Ordering
def selectKVerticeRandom (k:Int)(graph:Graph[Boolean,Float]):Graph[Boolean,Float] = {
    val graphWithRandomNodes = graph.mapVertices((vid,d) => scala.util.Random.nextFloat)
    val threshold = graphWithRandomNodes.vertices.map(v=>v._2).takeOrdered(k)(Ordering.by[Float,Float](t=>t)).last
    return graphWithRandomNodes.mapVertices((vid,randomVal) => randomVal <= threshold)
}

import scala.math.Ordering
selectKVerticeRandom: (k: Int)(graph: org.apache.spark.graphx.Graph[Boolean,Float])org.apache.spark.graphx.Graph[Boolean,Float]


In [7]:
def randomRange(a:Float,b:Float):Float = a + (b-a)*scala.util.Random.nextFloat

randomRange: (a: Float, b: Float)Float


## Données

### Karaté

In [8]:
val karate = GraphLoader.edgeListFile(sc,"data/soc-karate.mtx")
                .mapEdges(e => scala.util.Random.nextFloat)
                .mapVertices((vid,data) =>  false)

karate: org.apache.spark.graphx.Graph[Boolean,Float] = org.apache.spark.graphx.impl.GraphImpl@4dc1b40


### Facebook

In [9]:
val fb = GraphLoader.edgeListFile(sc,"data/facebook_combined.txt")

val edges = fb.edges.flatMap(e => Array(Edge(e.srcId,e.dstId,1f),Edge(e.dstId,e.srcId,1f))).distinct
val facebook = normalize_graph_weights(Graph(fb.vertices,edges)).mapVertices((vid,i)=>false)

fb: org.apache.spark.graphx.Graph[Int,Int] = org.apache.spark.graphx.impl.GraphImpl@16195722
edges: org.apache.spark.rdd.RDD[org.apache.spark.graphx.Edge[Float]] = MapPartitionsRDD[31] at distinct at <console>:37
facebook: org.apache.spark.graphx.Graph[Boolean,Float] = org.apache.spark.graphx.impl.GraphImpl@24e88f43


### Bitcoin

In [10]:
val edges =  sc.textFile("data/soc-sign-bitcoinotc.csv")
                .map(f => f.split(","))
                .map(a => Edge(a(0).toLong, a(1).toLong, a(2).toLong / 10f))
                .filter(e => (e.attr > 0))

val vertices = edges.flatMap(e => Array((e.srcId, false),(e.dstId,false))).distinct()

val bitcoin = Graph(vertices,edges)

edges: org.apache.spark.rdd.RDD[org.apache.spark.graphx.Edge[Float]] = MapPartitionsRDD[74] at filter at <console>:38
vertices: org.apache.spark.rdd.RDD[(org.apache.spark.graphx.VertexId, Boolean)] = MapPartitionsRDD[78] at distinct at <console>:40
bitcoin: org.apache.spark.graphx.Graph[Boolean,Float] = org.apache.spark.graphx.impl.GraphImpl@4fb045e8


## Tests

In [11]:
def test_IC(graph:Graph[Boolean,Float],selectFunc:Function[Graph[Boolean,Float],Graph[Boolean,Float]]):Long = {
    val graph_init = selectFunc(graph).mapVertices((vid,selected) =>  if (selected) "infected" else "susceptible")
    independant_cascade(graph_init).vertices.filter(v => v._2 != "susceptible").count
}

test_IC: (graph: org.apache.spark.graphx.Graph[Boolean,Float], selectFunc: Function[org.apache.spark.graphx.Graph[Boolean,Float],org.apache.spark.graphx.Graph[Boolean,Float]])Long


In [12]:
def test_LT(graph:Graph[Boolean,Float],selectFunc:Function[Graph[Boolean,Float],Graph[Boolean,Float]]):Long = {
    val graph_init = selectFunc(graph)
                        .mapVertices((vid,selected) => if (selected) 
                                                             ("infected",1f, randomRange(0.3f,0.9f)) 
                                                       else ("susceptible",0f, randomRange(0.3f,0.9f)))
    val normalized_graph_init = normalize_graph_weights(graph_init)
    linear_threshhold(normalized_graph_init).vertices.filter(v => v._2._1 != "susceptible").count
}

test_LT: (graph: org.apache.spark.graphx.Graph[Boolean,Float], selectFunc: Function[org.apache.spark.graphx.Graph[Boolean,Float],org.apache.spark.graphx.Graph[Boolean,Float]])Long


In [13]:
val facebook_IC_degrees = Array(1,2,5,10,20,30,50,100).map(k=>test_IC(facebook,selectKVerticesWithMaxDegrees(k)))
val facebook_IC_random = Array(1,2,5,10,20,30,50,100).map(k=>test_IC(facebook,selectKVerticeRandom(k)))
val facebook_LT_degrees = Array(1,2,5,10,20,30,50,100).map(k=>test_LT(facebook,selectKVerticesWithMaxDegrees(k)))
val facebook_LT_random = Array(1,2,5,10,20,30,50,100).map(k=>test_LT(facebook,selectKVerticeRandom(k)))
val bitcoin_IC_degrees = Array(1,2,5,10,20,30,50,100).map(k=>test_IC(bitcoin,selectKVerticesWithMaxDegrees(k)))
val bitcoin_IC_random = Array(1,2,5,10,20,30,50,100).map(k=>test_IC(bitcoin,selectKVerticeRandom(k)))
val bitcoin_LT_degrees = Array(1,2,5,10,20,30,50,100).map(k=>test_LT(bitcoin,selectKVerticesWithMaxDegrees(k)))
val bitcoin_LT_random = Array(1,2,5,10,20,30,50,100).map(k=>test_LT(bitcoin,selectKVerticeRandom(k)))


facebook_IC_degrees: Array[Long] = Array(156, 287, 710, 793, 861, 827, 868, 1224)
facebook_IC_random: Array[Long] = Array(1, 40, 37, 30, 86, 186, 366, 709)
facebook_LT_degrees: Array[Long] = Array(19, 32, 87, 101, 122, 119, 151, 225)
facebook_LT_random: Array[Long] = Array(1, 2, 5, 10, 20, 30, 50, 101)
bitcoin_IC_degrees: Array[Long] = Array(1769, 1710, 1763, 1781, 1815, 1789, 1758, 1760)
bitcoin_IC_random: Array[Long] = Array(1807, 1707, 1748, 1706, 1798, 1771, 1789, 1858)
bitcoin_LT_degrees: Array[Long] = Array(360, 455, 627, 830, 1165, 1390, 2169, 3253)
bitcoin_LT_random: Array[Long] = Array(1, 2, 6, 21, 26, 44, 72, 169)
